<a href="https://colab.research.google.com/github/Niraj-Thorat/ML-MAJOR-FEB-ML-02-BM3/blob/main/Sentiment_analysis_model_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

In [5]:
import sklearn

In [54]:
!pip install scikit-learn==0.24.1

     |████████████████████████████████| 22.3MB 53.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [6]:
sklearn.__version__

'0.24.1'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df=pd.read_csv('/content/drive/MyDrive/ARP_2/Datasets/Semantic_Data/Tweets.csv')

In [9]:
df.drop(columns=['tweet_id','airline_sentiment_confidence','negativereason','negativereason_confidence','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],inplace = True)

In [10]:
df_filtered=df[df['airline_sentiment'] != 'neutral']
df_filtered.head(20)

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,positive,"@VirginAmerica it was amazing, and arrived an ..."
11,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...
12,positive,@VirginAmerica This is such a great deal! Alre...
13,positive,@VirginAmerica @virginmedia I'm flying your #f...


In [11]:
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove("no")
stopword_list.remove("not")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
!pip install contractions

In [13]:
import requests
from bs4 import BeautifulSoup

In [14]:
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

In [15]:
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

In [16]:
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text

In [17]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

In [19]:
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip()for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

In [20]:
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(lambda x:x.lower())
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(html_tag)
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(con)
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(remove_sp)
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(remove_stopwords)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
df_filtered['airline_sentiment'].value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

In [22]:
x = df_filtered.iloc[:,1]
y = df_filtered.iloc[:,0]

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0,stratify=y)

In [24]:
print(x_train.shape)
print(x_test.shape)

(8078,)
(3463,)


In [26]:
import numpy as np
np.unique(y_train,return_counts=True)

(array(['negative', 'positive'], dtype=object), array([6424, 1654]))

In [27]:
np.unique(y_test,return_counts=True)

(array(['negative', 'positive'], dtype=object), array([2754,  709]))

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
sa_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())])

In [29]:
sa_model.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('model', SVC())])

In [30]:
y_pred = sa_model.predict(x_test)
y_pred

array(['negative', 'negative', 'negative', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [31]:
y_test

3512     negative
10159    negative
9752     negative
2876     negative
14311    negative
           ...   
4709     positive
10002    negative
6425     negative
1646     positive
2576     positive
Name: airline_sentiment, Length: 3463, dtype: object

In [32]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [33]:
accuracy_score(y_pred,y_test)*100

91.48137453075368

In [34]:
confusion_matrix(y_pred,y_test)

array([[2725,  266],
       [  29,  443]])

In [35]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    negative       0.99      0.91      0.95      2991
    positive       0.62      0.94      0.75       472

    accuracy                           0.91      3463
   macro avg       0.81      0.92      0.85      3463
weighted avg       0.94      0.91      0.92      3463



In [36]:
sentiment_m = df[df['airline_sentiment']=='negative']
sentiment_m.iloc[6].values

array(['negative',
       '@VirginAmerica you guys messed up my seating.. I reserved seating with my friends and you guys gave my seat away ... 😡 I want free internet'],
      dtype=object)

In [37]:
sa_model.predict(['@VirginAmerica you guys messed up my seating.. I reserved seating with my friends and you guys gave my seat away ... 😡 I want free internet'])

array(['negative'], dtype=object)

In [38]:
!pip install pipreqs

In [44]:
import joblib
joblib.dump(sa_model,'Tweets1')

['Tweets1']

In [40]:
!cp /content/Tweets /content/drive/MyDrive/ARP_2 -r

In [45]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

In [46]:
%%writefile Sentiments1.py
import streamlit as st
import sklearn
import joblib
model = joblib.load('Tweets')
st.title('Tweets Classifier')
ip = st.text_input('Enter your message')
op = model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

Overwriting Sentiments1.py


In [50]:
!nohup streamlit run Sentiments.py &
url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://3c6b41275e7c.ngrok.io'

In [43]:
sklearn.__version__

'0.24.1'

In [51]:
!pipreqs /content/

INFO: Successfully saved requirements file in /content/requirements.txt
